In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install sacremoses

     |████████████████████████████████| 880 kB 34.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c51a94a1d9d9329336fe8a984a3ed8c1180ae798fd96b8ad535552233c4d8e2d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [3]:
!pip install flair==0.10
!pip install seqeval

     |████████████████████████████████| 322 kB 34.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 981 kB 48.5 MB/s 
     |████████████████████████████████| 788 kB 56.9 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 4.2 MB 54.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.9 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 48 kB 3.3 MB/s 
     |████████████████████████████████| 19.7 MB 1.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 18.3 MB/s 
     |████████████████████████████████| 6.6 MB 36.3 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9692 sha256=5f1b89c7cc5cd2dc85cf8f704430cc5b361b7d06a8a7b6f

In [4]:
!pip install fasttext

     |████████████████████████████████| 68 kB 7.1 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3147819 sha256=df885f2f9a067c89e5a8f49a06f5e806709a0cda4c6b28bf019109d04223ad3d
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [5]:
# !unrar x -Y "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [6]:
import pandas as pd
import re
import numpy as np

In [7]:
ROOT = "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen"
MODEL = "BiLSTM-CRF-glove"
EMBEDDINGS = "w2vec_glove_wiki_id"
FINAL_MODEL_NAME = "glove"

In [8]:
# !unrar x "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/GloVe.rar" "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/"

In [9]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0: 'kata', 1: 'label'}

# this is the folder in which train, test and dev files reside

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(f"{ROOT}/nested", columns,
                              train_file='flair-train.txt',
                              test_file='flair-test.txt',
                              dev_file='flair-test.txt')

2022-05-19 10:02:56,945 Reading data from /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested
2022-05-19 10:02:56,946 Train: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-train.txt
2022-05-19 10:02:56,950 Dev: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt
2022-05-19 10:02:56,953 Test: /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/flair-test.txt


In [10]:
print(corpus.train[0].to_tagged_string('label'))

Dan <O+O> jika <O+O> orang <O+O> yang <O+O> berhutang <O+O> itu <O+O> dalam <O+O> kesukaran <O+O> maka <O+O> berilah <O+O> tangguh <O+O> sampai <O+O> dia <O+O> berkelapangan <O+O> Dan <O+O> menyedekahkan <O+O> sebagian <O+O> atau <O+O> semua <O+O> utang <O+O> itu <O+O> lebih <O+O> baik <O+O> bagimu <O+O> jika <O+O> kamu <O+O> mengetahui <O+O>


In [11]:
tag_type = 'label'

tag_dictionary = corpus.make_label_dictionary(tag_type)

2022-05-19 10:03:03,382 Computing label dictionary. Progress:


100%|██████████| 763/763 [00:00<00:00, 3799.50it/s]

2022-05-19 10:03:03,602 Corpus contains the labels: label (#29729)
2022-05-19 10:03:03,604 Created (for label 'label') Dictionary with 5 tags: <unk>, O+O, B-PER+B-PER, I-PER+I-PER, O+I-PER


In [12]:
print(tag_dictionary)

Dictionary with 5 tags: <unk>, O+O, B-PER+B-PER, I-PER+I-PER, O+I-PER


In [13]:
import gensim
import fasttext
import fasttext.util

In [14]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
 
# namaFileGlove = "/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/glove_wiki_id_300.txt"
# glove_file = datapath(namaFileGlove)
# tmp_file = get_tmpfile("/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_glove_wiki_id.txt")
 
# glove2word2vec(glove_file, tmp_file)

# word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_glove_wiki_id.txt', binary=False)
# word_vectors.save('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_glove_wiki_id')

In [15]:
from flair.embeddings import FastTextEmbeddings, WordEmbeddings

embedding = WordEmbeddings(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/{EMBEDDINGS}')

In [16]:
from flair.data import Sentence
embedding.embed(Sentence('12'))

[Sentence: "12"   [− Tokens: 1]]

In [17]:
# import fasttext.util
# ft = fasttext.load_model('/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/wiki.id.bin')
# ft.get_word_vector('12')

In [18]:
import requests, os
session = requests.Session()
session.verify = False
session.trust_env = False
os.environ['CURL_CA_BUNDLE']="" # or whaever other is interfering with 

from flair.models import SequenceTagger
from flair.embeddings import WordEmbeddings, FastTextEmbeddings
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embedding,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

SequenceTagger(
  (embeddings): WordEmbeddings(
    '/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_glove_wiki_id'
    (embedding): Embedding(481935, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=7, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [19]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
history = trainer.train(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}',
              learning_rate=0.1,
              max_epochs=200,
              )

2022-05-19 10:03:37,913 ----------------------------------------------------------------------------------------------------
2022-05-19 10:03:37,919 Model: "SequenceTagger(
  (embeddings): WordEmbeddings(
    '/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/embeddings/w2vec_glove_wiki_id'
    (embedding): Embedding(481935, 300)
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=300, out_features=300, bias=True)
  (rnn): LSTM(300, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=7, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)"
2022-05-19 10:03:37,920 ----------------------------------------------------------------------------------------------------
2022-05-19 10:03:37,923 Corpus: "Corpus: 763 train + 763 dev + 763 test sentences"
2022-05-19 10:03:37,925 ----------------------------------------------------------------------------------------------------
20

/usr/local/lib/python3.7/dist-packages/flair/trainers/trainer.py:65: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  "There should be no best model saved at epoch 1 except there is a model from previous trainings"


2022-05-19 10:03:38,590 ----------------------------------------------------------------------------------------------------
2022-05-19 10:03:39,052 epoch 1 - iter 2/24 - loss 1.84744472 - samples/sec: 139.99 - lr: 0.100000
2022-05-19 10:03:39,339 epoch 1 - iter 4/24 - loss 1.21564520 - samples/sec: 225.87 - lr: 0.100000
2022-05-19 10:03:39,720 epoch 1 - iter 6/24 - loss 0.90254385 - samples/sec: 169.84 - lr: 0.100000
2022-05-19 10:03:39,993 epoch 1 - iter 8/24 - loss 0.76595751 - samples/sec: 238.07 - lr: 0.100000
2022-05-19 10:03:40,445 epoch 1 - iter 10/24 - loss 0.66941525 - samples/sec: 142.64 - lr: 0.100000
2022-05-19 10:03:40,763 epoch 1 - iter 12/24 - loss 0.61134749 - samples/sec: 204.11 - lr: 0.100000
2022-05-19 10:03:41,038 epoch 1 - iter 14/24 - loss 0.57682091 - samples/sec: 237.44 - lr: 0.100000
2022-05-19 10:03:41,409 epoch 1 - iter 16/24 - loss 0.54395490 - samples/sec: 173.81 - lr: 0.100000
2022-05-19 10:03:41,739 epoch 1 - iter 18/24 - loss 0.51609878 - samples/sec: 1

In [20]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the trained model
model = SequenceTagger.load(f'/gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-{FINAL_MODEL_NAME}/best-model.pt')

2022-05-19 11:05:11,105 loading file /gdrive/MyDrive/kuliah/Tugas Akhir/Eksperimen/nested/results-model/final-model-glove/best-model.pt


In [21]:
# create example sentence
sentence = Sentence('yaitu mereka orang-orang yang dimurkai allah')
# predict the tags
model.predict(sentence)

print(sentence.to_tagged_string())

yaitu <O+O> mereka <O+O> orang-orang <B-PER+B-PER> yang <I-PER+I-PER> dimurkai <I-PER+I-PER> allah <I-PER+I-PER>


In [22]:
for entity in sentence.get_spans('label'):
    print(entity)

Span [1]: "yaitu"   [− Labels: O+O (0.9648)]
Span [2]: "mereka"   [− Labels: O+O (0.9435)]
Span [3,4,5,6]: "orang-orang yang dimurkai allah"   [− Labels: PER+I-PER (0.7021)]


In [23]:
sentence.to_dict('label')

{'entities': [{'end_pos': 5,
   'labels': [O+O (0.9648)],
   'start_pos': 0,
   'text': 'yaitu'},
  {'end_pos': 12, 'labels': [O+O (0.9435)], 'start_pos': 6, 'text': 'mereka'},
  {'end_pos': 44,
   'labels': [PER+I-PER (0.7021)],
   'start_pos': 13,
   'text': 'orang-orang yang dimurkai allah'}],
 'labels': [],
 'text': 'yaitu mereka orang-orang yang dimurkai allah'}

In [24]:
import pandas as pd
t = pd.read_csv(f"{ROOT}/nested/train_data.csv")
t

,juz,no_ayat,sent,tag,pos,sentence #
0,2,280,"['Dan', 'jika', 'orang', 'yang', 'berhutang', ...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['CC', 'SC', 'NN', 'SC', 'NN', 'PR', 'IN', 'NN...",287
1,5,31,"['Kemudian', 'Allah', 'menyuruh', 'seekor', 'b...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['NNP', 'NNP', 'CD', 'NND', 'NN', 'RB', 'VB', ...",700
2,2,97,"['Katakanlah', 'Barang', 'siapa', 'yang', 'men...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['NNP', 'NNP', 'WH', 'SC', 'VB', 'NN', 'NNP', ...",104
3,4,10,"['Sesungguhnya', 'orang-orang', 'yang', 'memak...","['O+O', 'B-PER+B-PER', 'I-PER+I-PER', 'I-PER+I...","['RB', 'NN', 'SC', 'VB', 'NN', 'NN', 'NN', 'IN...",503
4,2,29,"['Dialah', 'Allah', 'yang', 'menjadikan', 'seg...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['NNP', 'NNP', 'SC', 'VB', 'CD', 'SC', 'VB', '...",36
...,...,...,...,...,...,...
758,2,271,"['Jika', 'kamu', 'menampakkan', 'sedekahmu', '...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['SC', 'VB', 'VB', 'NN', 'SC', 'PR', 'VB', 'JJ...",278
759,6,29,"['Dan', 'tentu', 'mereka', 'akan', 'mengatakan...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['CC', 'RB', 'PRP', 'MD', 'VB', 'RB', 'NN', 'N...",818
760,2,249,"['Maka', 'tatkala', 'Thalut', 'keluar', 'memba...","['O+O', 'O+O', 'B-PER+B-PER', 'O+O', 'O+O', 'O...","['NNP', 'NNP', 'NNP', 'VB', 'VB', 'RB', 'PRP',...",256
761,3,28,"['Janganlah', 'orang-orang', 'mukmin', 'mengam...","['O+O', 'B-PER+B-PER', 'I-PER+I-PER', 'O+O', '...","['NN', 'NN', 'NN', 'VB', 'NN', 'NN', 'VB', 'NN...",321


In [25]:
t

,juz,no_ayat,sent,tag,pos,sentence #
0,2,280,"['Dan', 'jika', 'orang', 'yang', 'berhutang', ...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['CC', 'SC', 'NN', 'SC', 'NN', 'PR', 'IN', 'NN...",287
1,5,31,"['Kemudian', 'Allah', 'menyuruh', 'seekor', 'b...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['NNP', 'NNP', 'CD', 'NND', 'NN', 'RB', 'VB', ...",700
2,2,97,"['Katakanlah', 'Barang', 'siapa', 'yang', 'men...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['NNP', 'NNP', 'WH', 'SC', 'VB', 'NN', 'NNP', ...",104
3,4,10,"['Sesungguhnya', 'orang-orang', 'yang', 'memak...","['O+O', 'B-PER+B-PER', 'I-PER+I-PER', 'I-PER+I...","['RB', 'NN', 'SC', 'VB', 'NN', 'NN', 'NN', 'IN...",503
4,2,29,"['Dialah', 'Allah', 'yang', 'menjadikan', 'seg...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['NNP', 'NNP', 'SC', 'VB', 'CD', 'SC', 'VB', '...",36
...,...,...,...,...,...,...
758,2,271,"['Jika', 'kamu', 'menampakkan', 'sedekahmu', '...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['SC', 'VB', 'VB', 'NN', 'SC', 'PR', 'VB', 'JJ...",278
759,6,29,"['Dan', 'tentu', 'mereka', 'akan', 'mengatakan...","['O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+O', 'O+...","['CC', 'RB', 'PRP', 'MD', 'VB', 'RB', 'NN', 'N...",818
760,2,249,"['Maka', 'tatkala', 'Thalut', 'keluar', 'memba...","['O+O', 'O+O', 'B-PER+B-PER', 'O+O', 'O+O', 'O...","['NNP', 'NNP', 'NNP', 'VB', 'VB', 'RB', 'PRP',...",256
761,3,28,"['Janganlah', 'orang-orang', 'mukmin', 'mengam...","['O+O', 'B-PER+B-PER', 'I-PER+I-PER', 'O+O', '...","['NN', 'NN', 'NN', 'VB', 'NN', 'NN', 'VB', 'NN...",321


In [26]:
from ast import literal_eval

test_data = pd.read_csv(f"{ROOT}/nested/test_data.csv")

test_data['sent'] = test_data['sent'].apply(literal_eval)
test_data['tag'] = test_data['tag'].apply(literal_eval)
test_data['pos'] = test_data['pos'].apply(literal_eval)

ii_lab = []
for i, d in test_data.iterrows() :
  sent = ' '.join(d.sent)

  sentence = Sentence(sent)

  model.predict(sentence)

  t = sentence.to_tagged_string()

  word = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][::2] 
  tag = [i.strip() for i in re.split('\<(.*?)\>', t)][:-1][1::2] 
  
  ii_lab.append(tag)


In [27]:
test_data['pred'] = ii_lab
y_pred = ii_lab
y_true = test_data['tag'].to_list()

In [28]:
def get_level_token(seqs, max_lev=2, level=0) : 
  new_all_level = []
  for seq in seqs : 
    new_all_level_i = []
    for tag in seq : 
      new_tag = []
      tag_per_level = tag.split('+')
      for lev in range(max_lev) :
        if lev + 1 > len(tag_per_level) : 
          new_tag.append(tag_per_level[-1])
        else : 
          new_tag.append(tag_per_level[lev])
      new_all_level_i.append(new_tag)
    new_all_level.append(new_all_level_i)
  
  new_seq = []
  for seq in new_all_level :
    seq_i = [] 
    for tag in seq : 
      seq_i.append(tag[level])
    new_seq.append(seq_i)
  
  return new_seq

y_true_0 = get_level_token(y_true, max_lev=2, level=0)
y_pred_0 = get_level_token(y_pred, max_lev=2, level=0)

In [29]:
from seqeval.metrics import classification_report
print(classification_report(y_true_0, y_pred_0))

              precision    recall  f1-score   support

         PER       0.84      0.72      0.77       222

   micro avg       0.84      0.72      0.77       222
   macro avg       0.84      0.72      0.77       222
weighted avg       0.84      0.72      0.77       222



In [30]:
y_true_1 = get_level_token(y_true, max_lev=2, level=1)
y_pred_1 = get_level_token(y_pred, max_lev=2, level=1)

from seqeval.metrics import classification_report
print(classification_report(y_true_1, y_pred_1))

              precision    recall  f1-score   support

         PER       0.75      0.65      0.70       222

   micro avg       0.75      0.65      0.70       222
   macro avg       0.75      0.65      0.70       222
weighted avg       0.75      0.65      0.70       222



In [31]:
y_test_token = []
for i in y_true : 
  y_test_token = y_test_token + i

y_pred_token = []
for i in y_pred : 
  y_pred_token = y_pred_token + i

from sklearn.metrics import classification_report
print(classification_report(y_test_token, y_pred_token))


              precision    recall  f1-score   support

 B-PER+B-PER       0.91      0.78      0.84       222
 I-PER+I-PER       0.89      0.67      0.77       444
     O+I-PER       1.00      0.08      0.15       114
         O+O       0.95      0.99      0.97      6243

    accuracy                           0.95      7023
   macro avg       0.94      0.63      0.68      7023
weighted avg       0.95      0.95      0.94      7023

